# Neo4j

Image: https://hub.docker.com/_/neo4j

docker run \
    --publish=7474:7474 --publish=7687:7687 \
    --volume=$HOME/neo4j/data:/data
    --env=NEO4J_AUTH=none \
    neo4j

Relationship with props: https://pythontic.com/database/neo4j/create%20nodes%20and%20relationships 

Handbook: https://neo4j-contrib.github.io/py2neo/index.html#guides

GUI (web): http://localhost:7474/browser/

Clear DB:

```
MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r
```

In [1]:
import json
from elasticsearch import Elasticsearch
from py2neo import Graph, Node, Relationship

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('error', category=DeprecationWarning)

In [3]:
client = Elasticsearch([{"host": "127.0.0.1", "scheme": "http", "port": 9200}])

client.ping()

True

In [4]:
graph_db = Graph("bolt://localhost:7687", user='neo4j', password='12481632')

In [5]:
searchBody = {
  "size": 9999,
  "_source": True,
  "query": {
    "match_all": {}
  }
}

result = client.search(index="master", body=searchBody)

In [6]:
result['hits']['total']

{'value': 20, 'relation': 'eq'}

In [7]:
masters = result['hits']['hits']

In [8]:
len(masters)

20

In [9]:
masters[:3]

[{'_index': 'master',
  '_type': '_doc',
  '_id': '563268',
  '_score': 1.0,
  '_source': {'master_id': 563268,
   'master_desc': 'Милица Руслановна Фомина, Тип Образования: Среднее.',
   'master_feedbacks': ['ворчливый, медлительный.',
    'ворчливый, медлительный.',
    'медлительный, ворчливый.']}},
 {'_index': 'master',
  '_type': '_doc',
  '_id': '871908',
  '_score': 1.0,
  '_source': {'master_id': 871908,
   'master_desc': 'Козлов Юрий Фомич, Тип Образования: Среднее.',
   'master_feedbacks': ['ворчливый, аккуратный.',
    'медлительный, аккуратный.',
    'аккуратный, ворчливый.']}},
 {'_index': 'master',
  '_type': '_doc',
  '_id': '164475',
  '_score': 1.0,
  '_source': {'master_id': 164475,
   'master_desc': 'Юлий Дмитриевич Константинов, Тип Образования: Среднее.',
   'master_feedbacks': ['аккуратный, ворчливый.']}}]

In [12]:
# clear DB
graph_db.run("MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r;")
i = 0
for master in masters:
    try:
         print(f'{i})', end='')
         MasterNode = Node("Master",master_id=master['_source']['master_id'], master_desc=master['_source']['master_desc'])
         graph_db.create(MasterNode)
         searchBody = {
              "size": 9999,
              "_source": True,
              "query": {
                "term": {
                  "order_master_id": master['_source']['master_id']
               }
              }
            }
         result = client.search(index="order", body=searchBody)
         orders = result['hits']['hits']
         for order in orders:
             OrderNode = Node("Order",order_id=order['_source']['order_id'], order_date=order['_source']['order_date'], order_customer_desc=order['_source']['order_customer_desc'], order_price=order['_source']['order_price'])
             graph_db.create(OrderNode)
             NodesRelationship = Relationship(MasterNode, "Executed", OrderNode, order_due_date=order['_source']['order_due_date'], order_fact_completion_date=order['_source']['order_fact_completion_date'])
             graph_db.create(NodesRelationship)
             print('.', end='')
         print("+")
         i += 1
    except Exception as e:
        print(f'Exception: {e}')
        continue

0).......+
1)....+
2).........+
3).....+
4).......+
5)...+
6)...+
7).......+
8).....+
9)......+
10)..+
11)...+
12).....+
13)....+
14)......+
15)..+
16)...+
17).....+
18)....+
19)..........+


In [10]:
try:
    cur=graph_db.run("MATCH (MAS:Master)-[r:Executed]->(ORD:Order) WITH MAS, count(r) AS num RETURN MAS.master_desc as master_desc, num ORDER BY num DESC LIMIT 1;")
except Exception as e:
    print(e)

while cur.forward():
    print(cur.current)

'Воробьев Ювеналий Измаилович, Тип Образования: Среднее.'	10


In [11]:
searchBody = {
  "size": 9999,
  "_source": True,
  "query": {
    "match_all": {}
  }
}

result = client.search(index="order", body=searchBody)

In [12]:
result['hits']['hits']

[{'_index': 'order',
  '_type': '_doc',
  '_id': '131919',
  '_score': 1.0,
  '_source': {'order_id': 131919,
   'order_date': '2024-01-06',
   'order_customer_id': 227431,
   'order_customer_desc': 'Имя: Маргарита Валентиновна Чернова; премиальный, постоянный.',
   'order_details_desc': 'хрупкий, ценный.',
   'order_due_date': '2024-04-10',
   'order_fact_completion_date': '2024-04-27',
   'order_parts': ['аккумулятор'],
   'repair_types': ['замена компонентов'],
   'order_price': 2296.89215138,
   'order_master_id': 257458}},
 {'_index': 'order',
  '_type': '_doc',
  '_id': '664730',
  '_score': 1.0,
  '_source': {'order_id': 664730,
   'order_date': '2024-02-11',
   'order_customer_id': 656432,
   'order_customer_desc': 'Имя: Комарова Дарья Борисовна; постоянный, премиальный.',
   'order_details_desc': 'хрупкий, тяжелый.',
   'order_due_date': '2024-04-30',
   'order_fact_completion_date': '2024-04-20',
   'order_parts': ['дисплей', 'дисплей', 'usb-разъем'],
   'repair_types': ['мел